In [1]:
import os
from pathlib import Path
import re
import torch

from data_utils import embed_data_dir
from pytorch_lightning.callbacks import ModelCheckpoint
from mammo_net_density import *

In [2]:
train_csv_files = [
    ('/vol/biomedic3/bglocker/ugproj/vg521/model/mammo-net/csv/train/cf-and-real-embed-train.csv', 'cf_and_real'),
    ('/vol/biomedic3/bglocker/ugproj/vg521/model/mammo-net/csv/train/cf-embed-train.csv', 'cf_only'),
    ('/vol/biomedic3/bglocker/ugproj/vg521/model/mammo-net/csv/train/real-embed-train.csv', 'real_only')
]

train_csv_dict = {key: value for (value, key) in train_csv_files}

def extract_hparams_from_foldername(filename):
    filename = filename.strip()
    match = re.search(r'([^/]+)_bs(\d+)_lr([\d.]+)_epochs(\d+)', filename)
    if not match:
        print('checkpoint folder name does not match expected pattern')
    if match:
        name_model, batch_size, learning_rate, epochs = match.groups()
        
        parts = name_model.rsplit('_', 1)
        name, model = parts

        batch_size = int(batch_size)
        learning_rate = float(learning_rate)
        epochs = int(epochs)
        return {
            'name': name,
            'train_csv': train_csv_dict[name],
            'model': model,
            'batch_size': int(batch_size),
            'learning_rate': float(learning_rate),
            'epochs': int(epochs),
            'seed': 42,
            'num_devices': 1,
            'num_workers': 6,
            'batch_alpha': 1.0
        }

In [ ]:
test_csv = '/vol/biomedic3/bglocker/ugproj/vg521/model/mammo-net/csv/test/local-balanced-3397-test.csv'

In [ ]:
import importlib
import mammo_net_density

importlib.reload(mammo_net_density)

results = []

test_name = 'balanced-embed'

ckpts_path = Path('/vol/biomedic3/bglocker/ugproj/vg521/model/mammo-net/final_models')
# torch.set_float32_matmul_precision('medium')
torch.set_float32_matmul_precision('high')
                                       
# sets seeds for numpy, torch, python.random and PYTHONHASHSEED.
pl.seed_everything(seed=42, workers=True)

for ckpt_path in ckpts_path.iterdir():

    hparams = extract_hparams_from_foldername(os.path.basename(ckpt_path))
    
    if ckpt_path.is_dir():
        print(f'testing model in: {ckpt_path}')

    output_dir = os.path.join(ckpt_path,'test_existing_model_test')

    # find the best checkpoint
    ckpt_files = list(Path(ckpt_path/'version_0'/'checkpoints').glob('*.ckpt'))
    if not ckpt_files:
        print(f"no checkpoint found for {ckpt_path}")
        continue
    ckpt_path = str(ckpt_files[0])  
    ckpt = torch.load(ckpt_path)['state_dict']
    
    ckpt = {key.replace('model.', ''): value for key, value in ckpt.items()}
    model = MammoNet(backbone='resnet50', learning_rate=hparams['learning_rate'], checkpoint=ckpt)
    
    data = EMBEDMammoDataModule(image_size=(224, 168),
                                    val_percent=0.2,
                                    batch_alpha=hparams['batch_alpha'],
                                    batch_size=hparams['batch_size'],
                                    train_csv=hparams['train_csv'],
                                    test_csv=test_csv_path,
                                    num_workers=hparams['num_workers']
                                    )
    
    trainer = pl.Trainer(
        max_epochs=hparams['epochs'],
        accelerator='auto',
        devices=hparams['num_devices'],
        precision='16-mixed',
        num_sanity_val_steps=0,
        callbacks=[
            ModelCheckpoint(monitor='val_auc', mode='max'), 
            TQDMProgressBar(refresh_rate=10) 
            ],
    )
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    model.to(device)

    print('TESTING...')
    test_result = trainer.test(model=model, datamodule=data, ckpt_path=ckpt_path)
    save_predictions(model=model, output_fname=os.path.join(ckpt_path, 'predictions-{test_name}.csv'))

    results.append({
        'name': hp
        'model': hparams['model'],
        'batch_size': hparams['batch_size'],
        'learning_rate': hparams['learning_rate'],
        'epochs': hparams['epochs'],
        'test_auc': test_result.get('test_auc', 'NA'),
        'test_loss': test_result.get('test_loss', 'NA')
    })


Seed set to 42


testing model in: /vol/biomedic3/bglocker/ugproj/vg521/model/mammo-net/final_models/real_only_resnet50_bs64_lr0.0001_epochs5
<All keys matched successfully>
Dataset size (2.6 GiB) fits in cache limit (4 GiB). Allocating space to cache all 18857 samples.


/vol/biomedic3/bglocker/ugproj/vg521/counterfactuals/cf-env/lib/python3.12/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /vol/biomedic3/bglocker/ugproj/vg521/counterfactuals ...
Using 16bit Automatic Mixed Precision (AMP)


Creating `WeightedFixedBatchSampler`...
Sample population absolute class sizes: [9247 9610]
Sample population relative class sizes: [0.49037493 0.50962507]
Target batch class distribution [0.5 0.5] using alpha=1.0
Rounded batch class distribution [0.5 0.5]
Expecting [32 32] samples of each class per batch, over 294 batches of size 64
Sampling rates: [1.01741105 0.97898023]
samples (train):  18857
samples (val):    4722
samples (test):   3397
[9247 9610]
[2433 2289]
[1813 1584]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at /vol/biomedic3/bglocker/ugproj/vg521/model/mammo-net/final_models/real_only_resnet50_bs64_lr0.0001_epochs5/version_0/checkpoints/epoch=1-step=588.ckpt


...

Creating `WeightedFixedBatchSampler`...
Sample population absolute class sizes: [9392 9476]
Sample population relative class sizes: [0.49777401 0.50222599]
Target batch class distribution [0.5 0.5] using alpha=1.0
Rounded batch class distribution [0.5 0.5]
Expecting [128 128] samples of each class per batch, over 73 batches of size 256
Sampling rates: [0.99488927 0.98607007]
samples (train):  18868
samples (val):    4711
samples (test):   3397
[9392 9476]
[2288 2423]
[1813 1584]
TESTING...


/vol/biomedic3/bglocker/ugproj/vg521/counterfactuals/cf-env/lib/python3.12/site-packages/pytorch_lightning/trainer/call.py:282: Be aware that when using `ckpt_path`, callbacks used to create the checkpoint need to be provided during `Trainer` instantiation. Please add the following callbacks: ["EarlyStopping{'monitor': 'val_auc', 'mode': 'max'}"].
/vol/biomedic3/bglocker/ugproj/vg521/counterfactuals/cf-env/lib/python3.12/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:362: The dirpath has changed from 'final_models/real_only_resnet50_bs256_lr0.0005_epochs5/version_0/checkpoints' to '/vol/biomedic3/bglocker/ugproj/vg521/model/mammo-net/lightning_logs/version_4/checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only `best_model_path` will be reloaded.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /vol/biomedic3/bglocker/ugproj/vg521/model/mammo-net/f

Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_auc            0.9762027263641357
        test_loss           0.19659198820590973
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
testing model in: /vol/biomedic3/bglocker/ugproj/vg521/model/mammo-net/final_models/cf_and_real_resnet50_bs64_lr0.0001_epochs5
<All keys matched successfully>
Dataset size (10.6 GiB) exceeds cache limit (4 GiB). Allocating space to cache 28532 / 75764 samples.


KeyboardInterrupt: 